In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(200_000_000_000)

import intake
import geopandas as gpd
import pandas as pd
import numpy as np
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis import geography_utils

from shared_utils import webmap_utils, gtfs_utils_v2, rt_dates, portfolio_utils, rt_utils
import branca

In [ ]:
from update_vars import analysis_date, GCS_FILE_PATH, INTERSECTION_BUFFER_METERS, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD

In [ ]:
gcsgp = GCSGeoPandas()

In [ ]:
analysis_date

In [ ]:
catalog = intake.open_catalog('catalog.yml')

## Rail/Ferry/BRT Major Stops

In [ ]:
ca = catalog.ca_boundary.read().to_crs(geography_utils.CA_NAD83Albers_m)

In [ ]:
rail_ferry_brt = catalog.rail_brt_ferry_stops.read()

In [ ]:
rail_ferry_brt = rail_ferry_brt.clip(ca)

In [ ]:
hqta_types = rail_ferry_brt.hqta_type.unique()

In [ ]:
cmap = branca.colormap.step.Accent_08

In [ ]:
color_dict = webmap_utils.categorical_cmap(cmap, hqta_types)

In [ ]:
color_dict

In [ ]:
rail_ferry_brt = rail_ferry_brt.assign(color = rail_ferry_brt.hqta_type.map(color_dict))

In [ ]:
webmap_utils.set_state_export(rail_ferry_brt, filename = 'rail_ferry_brt3', cache_seconds=0)

In [ ]:
rail_ferry_brt

## Analysis Segments and Key Stops

* note that we drop circuitous segments for ease of visualization

In [ ]:
hqta_segments = catalog.hqta_segments.read()

In [ ]:
path = f'{GCS_FILE_PATH}all_bus.parquet'

In [ ]:
path

In [ ]:
max_arrivals_by_stop = pd.read_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop.parquet")

In [ ]:
gdf = gcsgp.read_parquet(path)

stops = gcsgp.read_parquet(f"{GCS_FILE_PATH}stops_with_lookback.parquet")

stops = stops[['stop_id', 'stop_name', 'analysis_date',
      'schedule_gtfs_dataset_key', 'analysis_name', 'geometry']]

stops = stops.rename(columns={'geometry': 'stop_geometry'})

gdf = gdf.merge(stops, on = ['stop_id', 'schedule_gtfs_dataset_key'])
gdf = gdf[~gdf['circuitous_segment']]

In [ ]:
map1 = gdf.copy()[['route_id', 'stop_id', 'geometry',
   'fwd_azimuth_360', 'circuitous_segment', 'hq_transit_corr',
   'ms_precursor', 'analysis_name']]

In [ ]:
# Source - https://stackoverflow.com/a
# Posted by mkrieger1, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-08, License - CC BY-SA 4.0

azimuth_cmap = branca.colormap.LinearColormap(
        colors=list(branca.colormap.linear.viridis.colors) + list(reversed(branca.colormap.linear.viridis.colors)),
        vmin=0, vmax=360
)  # this will correctly show 0 and 360 as close together
azimuth_cmap.caption = '360-degree azimuth (heading)'
azimuth_cmap

In [ ]:
webmap_utils.export_legend(azimuth_cmap, 'azimuth_viridis_360a.svg', inner_labels=list(range(72, 360, 72)))

In [ ]:
segment_state = webmap_utils.set_state_export(map1, filename = 'hqta_segments', cache_seconds=0,
                                     map_title='HQTA Segments', overwrite=True, color_col='fwd_azimuth_360',
                                     cmap = azimuth_cmap, legend_url='https://storage.googleapis.com/calitp-map-tiles/azimuth_viridis_360a.svg')

In [ ]:
map2 = gdf.copy()[['stop_id', 'stop_name', 'am_max_trips_hr',
                  'pm_max_trips_hr', 'route_dir_count', 'analysis_name',
                  'stop_geometry']].set_geometry('stop_geometry')

In [ ]:
# map2['color'] = [cmap_dict['stop']] * len(map2)
map2['color'] = [(0,0,0)] * len(map2)

In [ ]:
segment_stop_state = webmap_utils.set_state_export(map2, filename = 'hqta_segment_key_stops', cache_seconds=0,
                             existing_state=segment_state, map_title='Key Stops and Segments', overwrite=True,
                                                  manual_centroid=[37.336813156889704, -121.88911054161129])

In [ ]:
segment_stop_state

## Intersecting Segments

In [ ]:
pairs = pd.read_parquet(f"{GCS_FILE_PATH}pairwise.parquet")

## Spatial Intersections

In [ ]:
intersect = gcsgp.read_parquet(f"{GCS_FILE_PATH}all_intersections.parquet")

In [ ]:
by_segment = intersect.dissolve(['hqta_segment_id']).reset_index(drop=False)

In [ ]:
by_segment['color'] = [(140, 0, 0)] * len(by_segment)

In [ ]:
segment_intersect_state = webmap_utils.set_state_export(by_segment, filename = 'hqta_intersection_areas', cache_seconds=0,
                             existing_state=segment_state, map_title='Segments with Intersections', overwrite=True,
                                                       manual_centroid=[37.336813156889704, -121.88911054161129])

In [ ]:
segment_intersect_state

## Intersection Buffers and Stop Groups

In [ ]:
by_segment.geometry = by_segment.buffer(INTERSECTION_BUFFER_METERS)

In [ ]:
major_bus_spatial = gcsgp.read_parquet(f"{GCS_FILE_PATH}major_stop_bus.parquet")

In [ ]:
intersect_buffered_state = webmap_utils.set_state_export(by_segment, filename = 'intersect_buffered', cache_seconds=0,
                           map_title='Intersecton Buffers', overwrite=True,
                                                         manual_centroid=[37.336813156889704, -121.88911054161129])

In [ ]:
major_bus_spatial['color'] = [(200, 200, 255)] * len(major_bus_spatial)

In [ ]:
intersect_major_state = webmap_utils.set_state_export(major_bus_spatial, filename = 'major_bus_spatial', cache_seconds=0,
                           existing_state=intersect_buffered_state, map_title='Buffered Intersections and Stop Groups', overwrite=True,
                                                         manual_centroid=[37.336813156889704, -121.88911054161129])

In [ ]:
intersect_major_state